In [1]:
pip install "ray[air]" transformers datasets accelerate evaluate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [7]:
# Filename: ray_lightweight_llm_finetune.py

import ray
from ray.train.torch import TorchTrainer
from ray.train.huggingface.transformers import prepare_trainer, RayTrainReportCallback
from ray.train import ScalingConfig, RunConfig
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
import evaluate
import torch

# Step 1: Define Tokenization
def tokenize_data(example, tokenizer, max_length=128):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=max_length)

# Step 2: Prepare Dataset
def prepare_dataset(tokenizer):
    dataset = load_dataset("imdb", split="train[:2000]")  # small subset for demo
    tokenized_dataset = dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True)
    tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
    return tokenized_dataset

# Step 3: Define Trainer Initialization Function
def train_loop_per_worker(config):
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

    train_dataset = config["train_dataset"]
    eval_dataset = config["eval_dataset"]
    tokenizer_name = config["tokenizer_name"]

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    model = AutoModelForSequenceClassification.from_pretrained(tokenizer_name, num_labels=2)

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        logging_dir="./logs"
    )

    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = logits.argmax(axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    trainer.train()


# Step 4: Launch Ray and Fine-Tune
if __name__ == "__main__":
  ray.shutdown()
  ray.init(ignore_reinit_error=True)

  model_name = "distilbert-base-uncased"  # Small, lightweight model
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  tokenized = prepare_dataset(tokenizer)

  # Use TorchTrainer instead of the undefined TransformersTrainer
  trainer = TorchTrainer(
        train_loop_per_worker=train_loop_per_worker,
        train_loop_config={
            "train_dataset": tokenized["train"],
            "eval_dataset": tokenized["test"],
            "tokenizer_name": model_name,
        },
        scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
        run_config=RunConfig(name="lightweight-llm-finetune")
    )

  result = trainer.fit()
  print("Training completed:", result)
  ray.shutdown()

2025-08-10 12:06:12,283	INFO worker.py:1918 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

2025-08-10 12:06:18,333	INFO data_parallel_trainer.py:339 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.



View detailed results here: /root/ray_results/lightweight-llm-finetune
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-08-10_12-06-02_859952_169/artifacts/2025-08-10_12-06-18/lightweight-llm-finetune/driver_artifacts`


(TrainTrainable pid=6775) 2025-08-10 12:06:24.934225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=6775) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(TrainTrainable pid=6775) E0000 00:00:1754827584.955168    6775 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=6775) E0000 00:00:1754827584.961439    6775 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=6775) W0000 00:00:1754827584.978784    6775 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(TrainTrainable pid=6775) W0000 00:00:1754827

...

(RayTrainWorker pid=6877) {'eval_loss': 0.00014874870248604566, 'eval_accuracy': 1.0, 'eval_runtime': 42.8055, 'eval_samples_per_second': 9.345, 'eval_steps_per_second': 0.584, 'epoch': 2.0}


100%|██████████| 200/200 [25:18<00:00,  7.59s/it]


(RayTrainWorker pid=6877) {'train_runtime': 1518.306, 'train_samples_per_second': 2.108, 'train_steps_per_second': 0.132, 'train_loss': 0.01456852912902832, 'epoch': 2.0}


2025-08-10 12:32:12,209	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/lightweight-llm-finetune' in 0.0150s.



Training completed after 0 iterations at 2025-08-10 12:32:12. Total running time: 25min 53s

Training completed: Result(
  metrics={},
  path='/root/ray_results/lightweight-llm-finetune/TorchTrainer_6bfc5_00000_0_2025-08-10_12-06-18',
  filesystem='local',
  checkpoint=None
)
